# Projeto Final MegaDados
--------------
Insper

Prof. Fábio Ayres

Membros: Martim F., Sabrina S., Leonardo M

## Proposta Inicial
-----

O objetivo principal deste projeto foi estudar a manipulação de dados em larga escala. Qualquer quantidade de dados que não cabe na RAM pode ser considerada "BigData".
Neste projeto, foi usada uma fração do web crawler de setembro (http://commoncrawl.org/) que continha apenas os sites brasileiros, com um total de apenas 65GB de 7T originais.

### Tradução dos Dados em Hipóteses
------
Com isso, pensamos em analisar a frequência de palavras em documentos e montar um bag of words (BOW) para a totalidade dos documentos. Sequencialmente, fizemos um filtro que pegava apenas os documentos em que o nome de cada um dos 26 estado do Brasil fosse mencionado, e montamos um BOW individual para cada.

Será que uma palavra (ω) é mais frequênte em textos em que "São Paulo" é mencionado? Por exemplo... "Bras"
Será que as preposições são mencionadas igualmente no BOW total e nos BOWs filtrados? Provavelmente "Tu" para "São Paulo" terá uma frequência bem menor que do total.

Assim, temos:

> Hipótese 0: freq ω (estado) = freq ω (total)

Em contrapartida, é possível que palavras tenham frequências totalmente divergêntes entre um estado e a visão geral dos documentos coletados:

> Hipótese 1: freq ω (estado) ≠  freq ω (total)




### Operacionalização e Teste das Hipóteses
--------------



In [1]:
import pickle
import pprint

with open('./Pickle/palavras_pvalue.pickle', 'rb') as handle:
    dic_pvalue = pickle.load(handle)

In [2]:
def get_fracao(value, size):
    l = []
    for n in range(size):
        l.append(value[n][2]/value[n][1])
        a = value[n][2]
        b = value[n][1]
        print(a, b, a/b)
        
    return l

In [3]:
import pandas as pd

dts = {}
for key, value in dic_pvalue.items():
    print(key)
    size = len(value)

    palavra, P_total, P_estado, P_value = zip(*value)
    dt = pd.DataFrame({
        'palavra' : palavra,
        'P_total' : P_total,
        'P_estado': P_estado,
        'P-value' : P_value,
        #'Fração'  : fracao
        'Fração' : [P_estado[n] / P_total[n] for n in range(size)]
        })
    dts[key] = dt


Teresina
Aracaju
Macapá
Salvador
Belo Horizonte
Goiânia
Recife
Manaus
Fortaleza
Porto Alegre
Belém
João Pessoa
Natal
Rio Branco
São Paulo
Porto Velho
Campo Grande
Boa Vista
Brasília
Cuiabá
Curitiba
Maceió
Palmas
Florianópolis
Vitória
São Luís
Rio de Janeiro


In [6]:
dt = dts['Rio de Janeiro']
#dt = dts['Rio de Janeiro']

In [7]:
dt[dt['P-value'] < 0.2].sort_values(by=['Fração'], ascending=False)

,palavra,P_total,P_estado,P-value,Fração
213365,iGfooter,0.000055,0.000176,6.267765e-02,3.183242
89924,subHome,0.000107,0.000340,3.301112e-04,3.183242
110533,component,0.000324,0.001031,0.000000e+00,3.179768
12781,IG,0.000044,0.000139,1.425191e-01,3.141381
93799,ig,0.000154,0.000484,3.357952e-07,3.139738
175716,submenu,0.000046,0.000144,1.306999e-01,3.135150
236167,nav,0.000141,0.000429,7.119435e-06,3.052322
62807,10px,0.000063,0.000192,4.419642e-02,3.040979
228367,20px,0.000055,0.000167,8.094638e-02,3.035182
73106,weight,0.000054,0.000164,8.790369e-02,3.017807
